<a href="https://colab.research.google.com/github/ErrQD0303/MachineLearning/blob/main/GAT_geometric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install torch-geometric==2.5.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.6 MB/s eta 0:00:00


In [3]:
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root=".", name="Cora")
data = dataset[0]

In [5]:
# 1. Import thư viện
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATv2Conv
from torch.nn import Linear, Dropout

In [14]:
# 2. Định nghĩa hàm accuracy()
def accuracy(y_pred, y_true):
  return torch.sum(y_pred == y_true) / len(y_true)

In [15]:
# 3. Định nghĩa class GAT đại diện cho mô hình GAT
class GAT(torch.nn.Module):
  def __init__(self, dim_in, dim_h, dim_out, heads=8):
    super().__init__()
    self.gat1 = GATv2Conv(dim_in, dim_h, heads=heads)
    self.gat2 = GATv2Conv(dim_h*heads, dim_out, heads=1)

  # Định nghĩa hàm tiến có sử dụng eLU và chuẩn hóa softmax
  def forward(self, x, edge_index):
    h = F.dropout(x, p=0.6, training=self.training)
    h = self.gat1(h, edge_index)
    h = F.elu(h)
    h = F.dropout(h, p=0.6, training=self.training)
    h = self.gat2(h, edge_index)
    return F.log_softmax(h, dim=1)

  # Định nghĩa hàm fit để huấn luyện mô hình
  def fit(self, data, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(self.parameters(), lr=0.01, weight_decay=0.01)
    self.train()
    for epoch in range(epochs+1):
      optimizer.zero_grad()
      out = self(data.x, data.edge_index)
      loss = criterion(out[data.train_mask], data.y[data.train_mask])
      acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
      loss.backward()
      optimizer.step()

      if(epoch % 20 == 0):
        val_loss = criterion(out[data.val_mask], data.y[data.val_mask])
        val_acc = accuracy(out[data.val_mask].argmax(dim=1), data.y[data.val_mask])
        print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>5.2f}% | Val Loss: {val_loss:.2f} | Val Acc: {val_acc*100:.2f}%')

  # Định nghĩa hàm đánh giá test
  @torch.no_grad()
  def test(self, data):
    self.eval()
    out = self(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [16]:
# 3. Tiến hành train
gat = GAT(dataset.num_features, 32, dataset.num_classes)
gat.fit(data, epochs=100)

Epoch   0 | Train Loss: 1.962 | Train Acc: 14.29% | Val Loss: 1.96 | Val Acc: 14.00%
Epoch  20 | Train Loss: 0.210 | Train Acc: 96.43% | Val Loss: 0.92 | Val Acc: 72.40%
Epoch  40 | Train Loss: 0.217 | Train Acc: 97.14% | Val Loss: 0.91 | Val Acc: 73.40%
Epoch  60 | Train Loss: 0.177 | Train Acc: 99.29% | Val Loss: 0.90 | Val Acc: 72.00%
Epoch  80 | Train Loss: 0.193 | Train Acc: 97.86% | Val Loss: 0.99 | Val Acc: 71.40%
Epoch 100 | Train Loss: 0.206 | Train Acc: 97.14% | Val Loss: 0.91 | Val Acc: 72.00%


In [17]:
# Đánh giá dữ liệu sử dụng mô hình đã được train
acc = gat.test(data)
print(f'GAT test accuracy: {acc*100:.2f}%')

GAT test accuracy: 81.50%
